In [1]:
!pip install torchsummary wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 22.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 KB 8.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 KB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 KB 3.2 MB/s eta 0:00:00
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8807 sha256=5d367e084a7c45b5aa6c464ec1599061cd053ed3af13b3ac2ba76179a58d0211
  Stored in directory: /root/.cache/pip/wheels/b7/0a/67/ada2a22079218c75a88361c0782855cc72aebc4d18d0289d05
Successfully built pathtools


In [2]:
import torch
from torchsummary import summary
import torchvision #This library is used for image-based operations (Augmentations)
import wandb
import os
import gc
from tqdm import tqdm
import math
from PIL import Image
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score
import glob

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
print("Device: ", DEVICE)

Device:  cuda


In [ ]:
!pip install --upgrade --force-reinstall --no-deps kaggle==1.5.8
!mkdir /root/.kaggle

with open("/root/.kaggle/kaggle.json", "w+") as f:
    f.write('{"username":"<username>","key":"<key>"}') 
    # Put your kaggle username & key here

!chmod 600 /root/.kaggle/kaggle.json

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.2/59.2 KB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for kaggle: filename=kaggle-1.5.8-py3-none-any.whl size=73272 sha256=2e21d6e8d341c3468f211ac7512432801059e87f76424804ee04e9a19e47d9a8
  Stored in directory: /root/.cache/pip/wheels/d4/02/ef/3f8c8d86b8d5388a1d3155876837f1a1a3143ab3fc2ff1ffad
Successfully built kaggle
  Attempting uninstall: kaggle
    Found existing installation: kaggle 1.5.13
    Uninstalling kaggle-1.5.13:
      Successfully uninstalled kaggle-1.5.13


In [ ]:
!mkdir '/content/data'

#!kaggle competitions download -c 11-785-s23-hw2p2-classification
#!unzip -qo '11-785-s23-hw2p2-classification.zip' -d '/content/data'

!kaggle competitions download -c 11-785-s23-hw2p2-classification
!kaggle competitions download -c 11-785-s23-hw2p2-verification
!unzip -qo '11-785-s23-hw2p2-classification.zip' -d '/content/data'
!unzip -qo '11-785-s23-hw2p2-verification.zip' -d '/content/data'


100% 1.71G/1.72G [00:08<00:00, 319MB/s]
100% 1.72G/1.72G [00:08<00:00, 221MB/s]
 30% 5.00M/16.8M [00:00<00:00, 37.7MB/s]
100% 16.8M/16.8M [00:00<00:00, 102MB/s] 


In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
config = {
    'batch_size': 128, # Increase this if your GPU can handle it
    'lr': 0.01,
    'epochs': 80, # 10 epochs is recommended ONLY for the early submission - you will have to train for much longer typically.
    # Include other parameters as needed.
}

In [ ]:
DATA_DIR    = '/content/data/11-785-s23-hw2p2-classification'# TODO: Path where you have downloaded the data
TRAIN_DIR   = os.path.join(DATA_DIR, "train") 
VAL_DIR     = os.path.join(DATA_DIR, "dev")

# Transforms using torchvision - Refer https://pytorch.org/vision/stable/transforms.html

train_transforms = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor(), 
    torchvision.transforms.Normalize(mean = [0.5116, 0.4026, 0.3519], std = [0.3073, 0.2697, 0.2587])
])# Implementing the right train transforms/augmentation methods is key to improving performance.

# Most torchvision transforms are done on PIL images. So you convert it into a tensor at the end with ToTensor()
# But there are some transforms which are performed after ToTensor() : e.g - Normalization
# Normalization Tip - Do not blindly use normalization that is not suitable for this dataset

valid_transforms = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor(), 
    torchvision.transforms.Normalize(mean = [0.5116, 0.4026, 0.3519], std = [0.3073, 0.2697, 0.2587])
])


train_dataset   = torchvision.datasets.ImageFolder(TRAIN_DIR, transform= train_transforms)
valid_dataset   = torchvision.datasets.ImageFolder(VAL_DIR, transform= valid_transforms)
# You should NOT have data augmentation on the validation set. Why?


# Create data loaders
train_loader = torch.utils.data.DataLoader(
    dataset     = train_dataset, 
    batch_size  = config['batch_size'], 
    shuffle     = True,
    num_workers = 4, 
    pin_memory  = True
)

valid_loader = torch.utils.data.DataLoader(
    dataset     = valid_dataset, 
    batch_size  = config['batch_size'],
    shuffle     = False,
    num_workers = 2
)

In [ ]:
print("Number of classes    : ", len(train_dataset.classes))
print("No. of train images  : ", train_dataset.__len__())
print("Shape of image       : ", train_dataset[0][0].shape)
print("Batch size           : ", config['batch_size'])
print("Train batches        : ", train_loader.__len__())
print("Val batches          : ", valid_loader.__len__())

Number of classes    :  7000
No. of train images  :  140000
Shape of image       :  torch.Size([3, 224, 224])
Batch size           :  128
Train batches        :  1094
Val batches          :  274


# ArcFace Loss

[ArcFace: Additive Angular Margin Loss for Deep
Face Recognition](https://arxiv.org/pdf/1801.07698.pdf)

ArcFace Loss is trying to maximize the geodesic distance on the hypersphere between features of different classes to make the features more separately. Here is a blog that explains ArcFace Loss in detail: [link](https://medium.com/analytics-vidhya/face-recognition-and-arcface-additive-angular-margin-loss-for-deep-face-recognition-44abc56916c#:~:text=The%20ArcFace%20loss%20maximizes%20the,implemented%20with%20negligible%20computational%20overhead)

$$L_{afl} = - log \frac{e^{scos(\theta_{y_i} + m)}}{e^{s cos(\theta_{y_i} + m)} + \sum_{j=1,j \neq y_i}^N e^{s cos(\theta_j)}}$$



In [3]:
import copy
class ArcFaceModel(torch.nn.Module):
    '''
    To train in a standard training loop make sure to modify the train function so you pass in the inputs and the labels
    i.e. output = model(images, labels)
    '''
    def __init__(self, model, margin=0.5, scaler=30, embedding_size=512, num_classes=7000):
        super(ArcFaceModel, self).__init__()
        self.embedding_size = embedding_size
        self.num_classes = num_classes

        # small number to avoid invalid arcCos values
        self.eps = 1e-6
        
        # hyperparameters 
        self.margin = margin
        self.scaler = scaler

        # load classification model
        self.model = model

        # Initializing the arcface linear layer with the weights of the classifier from the trained CNN
        self.AFL_linear = torch.nn.Linear(embedding_size, num_classes, bias=False) # Why set bias=False? Check out the paper.
        with torch.no_grad():
          self.AFL_linear.weight.copy_(self.model.fc.weight)

        # Initializing utility functions for normalization, arcCos, cos and onehot encoding
        self.normalizer = torch.nn.functional.normalize
        self.arcCos = torch.acos
        self.cos = torch.cos
        self.one_hot = torch.nn.functional.one_hot

      
    def forward(self, x, labels):
        # Get face embedding
        embedding = self.model(x, return_feats=True)

        # TODO: normalize face embedding
        embedding = self.normalizer(embedding, p=2)

        # TODO: normalize linear layer weights
        with torch.no_grad():
          self.AFL_linear.weight = torch.nn.Parameter(self.normalizer(self.AFL_linear.weight, p=2), requires_grad=True)
        
        # TODO: take dot product to get cos theta, remember that Wx = ||W||||x||cos(\theta) and ||W|| = 1, ||x|| = 1
        
        cosine = torch.mm(embedding, self.AFL_linear.weight.T)
        
        # We clamp the values to be a little higher than -1 and a little lower than one so we don't get nan values when we call arccos
        cosine = torch.clamp(cosine, min=-1.0+self.eps, max=1.0-self.eps)
   
        # TODO: get theta by performing arccos(cos(theta))
        theta = self.arcCos(cosine)

        # TODO: convert labels to one-hot
        one_hot_labels = self.one_hot(labels, num_classes = self.num_classes)
        # TODO: create a mask with m at positions with label 1 and 0 at positions with label 0
        margin_mask = torch.where(one_hot_labels == 1, self.margin, 0)
        # TODO: add margin m to theta
        theta_m = theta + margin_mask

        # calculate the cosine value for theta with margin added and scale with self.scaler
        logits = self.scaler * self.cos(theta_m) # this value is then passed to crossEntropyLoss in train loop to calculate arcface loss
        return logits

In [4]:
# I have referenced https://blog.paperspace.com/writing-resnet-from-scratch-in-pytorch/ while building ResNet. 
# The layer names are similar for debugging purposes. 

class ResidualBlock(torch.nn.Module):
    def __init__(self, in_channels, out_channels, stride = 1, downsample=None):
        super(ResidualBlock, self).__init__()
        self.conv1 = torch.nn.Sequential(
            torch.nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=3,
                            stride = stride, padding = 1),
            torch.nn.BatchNorm2d(out_channels),
            torch.nn.ReLU()
        )
        self.conv2 = torch.nn.Sequential(
            torch.nn.Conv2d(in_channels=out_channels, out_channels=out_channels, kernel_size=3,
                                     stride = 1, padding=1),
            torch.nn.BatchNorm2d(out_channels)
        )

        self.downsample = downsample
        self.relu = torch.nn.ReLU()
        self.out_channels = out_channels


    def forward(self, x):
        output = self.conv1(x)
        output = self.conv2(output)
        if self.downsample:
            residual = self.downsample(x)
        else:
            residual = x
        output = output + residual
        output = self.relu(output)
        return output



class ResNet(torch.nn.Module):
    def __init__(self, block, layers, num_classes = 7000):
        super(ResNet, self).__init__()
        self.num_channels = 64
        self.num_classes = num_classes

        self.conv1 = torch.nn.Sequential(
            torch.nn.Conv2d(in_channels=3, out_channels=64, kernel_size=7, stride = 2, padding=3),
            torch.nn.BatchNorm2d(self.num_channels),
            torch.nn.ReLU())

        self.maxpool = torch.nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer0 = self.layer_blocks(block, 64, layers[0], stride=1)
        self.layer1 = self.layer_blocks(block, 128, layers[1], stride=2)
        self.layer2 = self.layer_blocks(block, 256, layers[2], stride=2)
        self.layer3 = self.layer_blocks(block, 512, layers[3], stride=2)
        self.avgpool = torch.nn.AvgPool2d(7, stride=1)
        self.fc = torch.nn.Linear(512, self.num_classes)


    def layer_blocks(self, block, num_filters, num_blocks, stride = 1):
        downsample = None
        if (stride != 1) or (num_filters != self.num_channels):
            downsample = torch.nn.Sequential(
                torch.nn.Conv2d(in_channels=self.num_channels, out_channels=num_filters, kernel_size=1, stride = stride),
                torch.nn.BatchNorm2d(num_filters)
            )
        else:
            downsample = None
        layers = [block(self.num_channels, num_filters, stride, downsample)]
        self.num_channels = num_filters

        for i in range(1, num_blocks):
            layers.append(block(self.num_channels, num_filters))

        return torch.nn.Sequential(*layers)


    def forward(self, x):
        x = self.conv1(x)
        x = self.maxpool(x)
        x = self.layer0(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)

        x = self.avgpool(x)

        x = x.view(x.size(0), -1)
        x = self.fc(x)

        return x


In [7]:
model = ResNet(ResidualBlock, layers = [3, 4, 6, 3])
checkpoint = torch.load('/content/drive/MyDrive/IDL/HW2P2/models/ResNet34-restart-2.pth', map_location=torch.device('cpu'))
model.load_state_dict(checkpoint['model_state_dict'])
arcFaceModel = ArcFaceModel(model, margin=0.5, scaler=64
                            , embedding_size=512, num_classes=7000).to(DEVICE)

In [ ]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=config['lr'])
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor = 0.2, patience = 2)
scaler = torch.cuda.amp.GradScaler()

In [ ]:
def train(model, dataloader, optimizer, criterion):
    
    model.train()

    # Progress Bar 
    batch_bar   = tqdm(total=len(dataloader), dynamic_ncols=True, leave=False, position=0, desc='Train', ncols=5) 

    num_correct = 0
    total_loss  = 0

    for i, (images, labels) in enumerate(dataloader):
        
        optimizer.zero_grad() # Zero gradients

        images, labels = images.to(DEVICE), labels.to(DEVICE)
        
        with torch.cuda.amp.autocast(): # This implements mixed precision. Thats it! 
            outputs = model(images, labels)
            loss    = criterion(outputs, labels)

        # Update no. of correct predictions & loss as we iterate
        num_correct     += int((torch.argmax(outputs, axis=1) == labels).sum())
        total_loss      += float(loss.item())

        # tqdm lets you add some details so you can monitor training as you train.
        batch_bar.set_postfix(
            acc         = "{:.04f}%".format(100 * num_correct / (config['batch_size']*(i + 1))),
            loss        = "{:.04f}".format(float(total_loss / (i + 1))),
            num_correct = num_correct,
            lr          = "{:.06f}".format(float(optimizer.param_groups[0]['lr']))
        )
        
        scaler.scale(loss).backward() # This is a replacement for loss.backward()
        scaler.step(optimizer) # This is a replacement for optimizer.step()
        scaler.update() 

        # TODO? Depending on your choice of scheduler,
        # You may want to call some schdulers inside the train function. What are these?
      
        batch_bar.update() # Update tqdm bar

    batch_bar.close() # You need this to close the tqdm bar

    acc         = 100 * num_correct / (config['batch_size']* len(dataloader))
    total_loss  = float(total_loss / len(dataloader))

    return acc, total_loss

In [ ]:
def validate(model, dataloader, criterion):
  
    model.eval()
    batch_bar = tqdm(total=len(dataloader), dynamic_ncols=True, position=0, leave=False, desc='Val', ncols=5)

    num_correct = 0.0
    total_loss = 0.0

    for i, (images, labels) in enumerate(dataloader):
        
        # Move images to device
        images, labels = images.to(DEVICE), labels.to(DEVICE)
        
        # Get model outputs
        with torch.inference_mode():
            outputs = model(images, labels)
            loss = criterion(outputs, labels)

        num_correct += int((torch.argmax(outputs, axis=1) == labels).sum())
        total_loss += float(loss.item())

        batch_bar.set_postfix(
            acc="{:.04f}%".format(100 * num_correct / (config['batch_size']*(i + 1))),
            loss="{:.06f}".format(float(total_loss / (i + 1))),
            num_correct=num_correct)

        batch_bar.update()
        
    batch_bar.close()
    acc = 100 * num_correct / (config['batch_size']* len(dataloader))
    total_loss = float(total_loss / len(dataloader))
    return acc, total_loss

In [ ]:
# This obtains the list of known identities from the known folder
known_regex = "/content/data/11-785-s23-hw2p2-verification/known/*/*"
known_paths = [i.split('/')[-2] for i in sorted(glob.glob(known_regex))]

# Obtain a list of images from unknown folders
unknown_dev_regex = "/content/data/11-785-s23-hw2p2-verification/unknown_dev/*"
unknown_test_regex = "/content/data/11-785-s23-hw2p2-verification/unknown_test/*"

# We load the images from known and unknown folders
unknown_dev_images = [Image.open(p) for p in tqdm(sorted(glob.glob(unknown_dev_regex)))]
unknown_test_images = [Image.open(p) for p in tqdm(sorted(glob.glob(unknown_test_regex)))]
known_images = [Image.open(p) for p in tqdm(sorted(glob.glob(known_regex)))]

# Why do you need only ToTensor() here?
transforms = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor()])

unknown_dev_images = torch.stack([transforms(x) for x in unknown_dev_images])
unknown_test_images = torch.stack([transforms(x) for x in unknown_test_images])
known_images  = torch.stack([transforms(y) for y in known_images ])
#Print your shapes here to understand what we have done

# You can use other similarity metrics like Euclidean Distance if you wish
similarity_metric = torch.nn.CosineSimilarity(dim= 1, eps= 1e-6) 

100%|██████████| 960/960 [00:00<00:00, 4469.38it/s]


In [ ]:
def eval_verification(unknown_images, known_images, model, similarity, batch_size= config['batch_size'], mode='val'): 

    unknown_feats, known_feats = [], []

    batch_bar = tqdm(total=len(unknown_images)//batch_size, dynamic_ncols=True, position=0, leave=False, desc=mode)
    model.eval()

    # We load the images as batches for memory optimization and avoiding CUDA OOM errors
    for i in range(0, unknown_images.shape[0], batch_size):
        unknown_batch = unknown_images[i:i+batch_size] # Slice a given portion upto batch_size
        
        with torch.no_grad():
            unknown_feat = model(unknown_batch.float().to(DEVICE), return_feats=True) #Get features from model         
        unknown_feats.append(unknown_feat)
        batch_bar.update()
    
    batch_bar.close()
    
    batch_bar = tqdm(total=len(known_images)//batch_size, dynamic_ncols=True, position=0, leave=False, desc=mode)
    
    for i in range(0, known_images.shape[0], batch_size):
        known_batch = known_images[i:i+batch_size] 
        with torch.no_grad():
              known_feat = model(known_batch.float().to(DEVICE), return_feats=True)
          
        known_feats.append(known_feat)
        batch_bar.update()

    batch_bar.close()

    # Concatenate all the batches
    unknown_feats = torch.cat(unknown_feats, dim=0)
    known_feats = torch.cat(known_feats, dim=0)

    similarity_values = torch.stack([similarity(unknown_feats, known_feature) for known_feature in known_feats])
    # Print the inner list comprehension in a separate cell - what is really happening?

    max_similarity_values, predictions = similarity_values.max(0) #Why are we doing an max here, where are the return values?
    max_similarity_values, predictions = max_similarity_values.cpu().numpy(), predictions.cpu().numpy()


    # Note that in unknown identities, there are identities without correspondence in known identities.
    # Therefore, these identities should be not similar to all the known identities, i.e. max similarity will be below a certain 
    # threshold compared with those identities with correspondence.

    # In early submission, you can ignore identities without correspondence, simply taking identity with max similarity value
    pred_id_strings = [known_paths[i] for i in predictions] # Map argmax indices to identity strings
    
    # After early submission, remove the previous line and uncomment the following code 

    threshold = 0.5
    NO_CORRESPONDENCE_LABEL = 'n000000'
    pred_id_strings = []
    for idx, prediction in enumerate(predictions):
        if max_similarity_values[idx] < threshold: # why < ? Thank about what is your similarity metric
            pred_id_strings.append(NO_CORRESPONDENCE_LABEL)
        else:
            pred_id_strings.append(known_paths[prediction])
  
    if mode == 'val':
      true_ids = pd.read_csv('/content/data/11-785-s23-hw2p2-verification/verification_dev.csv')['label'].tolist()
      accuracy = accuracy_score(pred_id_strings, true_ids)
      print("Verification Accuracy = {}".format(accuracy))

    return accuracy

In [ ]:
gc.collect() # These commands help you when you face CUDA OOM error
torch.cuda.empty_cache()

In [ ]:
wandb.login(key="") #API Key is in your wandb account, under settings (wandb.ai/settings)

wandb: Currently logged in as: krana (cmu-idl). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
# Create your wandb run
run = wandb.init(
    name = "ResNet34-arcFace-freeze", ## Wandb creates random run names if you skip this field
    reinit = True, ### Allows reinitalizing runs when you re-run this cell
    #id = 'vo24kiuj', #Insert specific run id here if you want to resume a previous run
    #resume = "must", ### You need this to resume previous runs, but comment out reinit = True when using this
    project = "hw2p2-krana", ### Project should be created in your wandb account 
    config = config ### Wandb Config for your run
)

In [ ]:
best_valacc = 0.0

for epoch in range(config['epochs']):

    curr_lr = float(optimizer.param_groups[0]['lr'])

    train_acc, train_loss = train(arcFaceModel, train_loader, optimizer, criterion)
    
    print("\nEpoch {}/{}: \nTrain Acc {:.04f}%\t Train Loss {:.04f}\t Learning Rate {:.06f}".format(
        epoch + 1,
        config['epochs'],
        train_acc,
        train_loss,
        curr_lr))
    
    val_acc, val_loss = validate(arcFaceModel, valid_loader, criterion)
    eval_acc = eval_verification(unknown_dev_images, known_images, model, similarity_metric, config['batch_size'], mode='val')

    
    print("Val Acc {:.04f}%\t Val Loss {:.04f} \t Eval Acc {:.04f}".format(val_acc, val_loss, eval_acc))

    wandb.log({"train_loss":train_loss, 'train_Acc': train_acc, 'validation_Acc':val_acc, 
               'validation_loss': val_loss, "learning_Rate": curr_lr})
    
    # If you are using a scheduler in your train function within your iteration loop, you may want to log
    # your learning rate differently 

    # #Save model in drive location if val_acc is better than best recorded val_acc
    if val_acc >= best_valacc:
      #path = os.path.join(root, model_directory, 'checkpoint' + '.pth')
      print("Saving model")
      torch.save({'model_state_dict':model.state_dict(),
                  'optimizer_state_dict':optimizer.state_dict(),
                  #'scheduler_state_dict':scheduler.state_dict(),
                  'val_acc': val_acc, 
                  'epoch': epoch}, '/content/drive/MyDrive/IDL/HW2P2/models-contrastive/ResNet34-arcface.pth')
      torch.save({'model_state_dict':model.state_dict(),
                  'optimizer_state_dict':optimizer.state_dict(),
                  #'scheduler_state_dict':scheduler.state_dict(),
                  'val_acc': val_acc, 
                  'epoch': epoch}, '/content/ResNet34-arcface.pth')
      best_valacc = val_acc
      wandb.save('ResNet34-arcFace.pth')
    scheduler.step(val_loss)
      # You may find it interesting to exlplore Wandb Artifcats to version your models
run.finish()


Epoch 1/80: 
Train Acc 0.0000%	 Train Loss 18.7650	 Learning Rate 0.010000


Verification Accuracy = 0.075
Val Acc 0.0000%	 Val Loss 18.7315 	 Eval Acc 0.0750
Saving model



Epoch 2/80: 
Train Acc 0.0000%	 Train Loss 18.7265	 Learning Rate 0.010000


Verification Accuracy = 0.1361111111111111
Val Acc 0.0000%	 Val Loss 18.7304 	 Eval Acc 0.1361
Saving model



Epoch 3/80: 
Train Acc 0.0000%	 Train Loss 18.7191	 Learning Rate 0.010000


...

Verification Accuracy = 0.12222222222222222
Val Acc 0.0000%	 Val Loss 18.6536 	 Eval Acc 0.1222
Saving model



Epoch 44/80: 
Train Acc 0.0000%	 Train Loss 18.4797	 Learning Rate 0.000000


Verification Accuracy = 0.13333333333333333
Val Acc 0.0000%	 Val Loss 18.6516 	 Eval Acc 0.1333
Saving model


Train:  72%|███████▏  | 787/1094 [01:08<00:26, 11.55it/s, acc=0.0000%, loss=18.4793, lr=0.000000, num_correct=0]

KeyboardInterrupt: ignored

# SphereFace Loss
[SphereFace: Deep Hypersphere Embedding for Face Recognition](https://arxiv.org/pdf/1704.08063.pdf)

[SphereFace Revived:
Unifying Hyperspherical Face Recognition](https://arxiv.org/pdf/2109.05565.pdf)

$$L_{sfl} = - log \frac{e^{scos(m\theta_{y_i})}}{e^{s cos(m\theta_{y_i})} + \sum_{j=1,j \neq y_i}^N e^{s cos(\theta_j)}}$$

Notice that the only difference between arcface loss and sphere loss is from $e^{scos(\theta_{y_i} + m)}$ to $e^{scos(m\theta_{y_i})}$. You should be able to implement this based on the comments in ArcFace loss and update `margin_mask` variable accordingly.

